#  Decision Tree on Google PlayStore Apps 

这次我想通过使用决策树对 __[Kaggle 的 Google PlayStore 应用数据集](https://www.kaggle.com/lava18/google-play-store-apps)__ 进行分析，希望找出与应用__安装量__密切相关的因素，也是希望在工作上对我所在的海外部门的应用市场数据增长起到一定作用。

## 初始化

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph.

import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import warnings
warnings.filterwarnings('ignore')
from pylab import rcParams
# figure size in inches

%matplotlib inline

ModuleNotFoundError: No module named 'numpy'

## 数据集初探

In [ ]:
df = pd.read_csv('data/googleplaystore.csv')
print('Number of apps in the dataset : ' , len(df))
df.sample(7)


可以看出数据集里对每一个应用共有 13 种属性的定义，根据 Title 就能很直观的明白其意义。
这里我将**安装量**(*Installs*)作为我们的目标，为了实现分类，可以将安装的数量按一定的数据区间分类：

| **安装量**         | **Label**         | 
| ------------- |---------------| 
| 0 - 1000    | 1 | 
| 1000 - 5000      | 2      |  
| 5000 - 10k | 3      |
| 10k - 50k | 4      | 
| 50k - 100k | 5      | 
| 100k - 500k | 6      | 
| 500k - 1m | 7      | 
| 1m - 5m | 8      | 
| 5m - 10m | 9      | 
| 10m - 50m | 10      | 
| 50m+ | 11      | 

考虑到目前我们海外版的应用安装量在 5 milliions 数量级，所以这里代表最高安装量的分类暂定为 50m+（个人觉得已经是个非常高的界限了）

## 数据预处理

In [ ]:
print( len(df['Installs'].unique()) , "Installs")
print("\n", df['Installs'].unique())

In [ ]:
# - Installs : Remove + and ,

df = df[df['Installs'] != 'Free']
df['Installs'] = df['Installs'].apply(lambda x: x.replace('+', '') if '+' in str(x) else x)
df['Installs'] = df['Installs'].apply(lambda x: x.replace(',', '') if ',' in str(x) else x)
df['Installs'] = df['Installs'].apply(lambda x: int(x))
print( len(df['Installs'].unique()) , "Installs")
print("\n", df['Installs'].unique())

In [ ]:
data = [go.Histogram(
        x = df.Rating,
        xbins = {'start': 1, 'size': 0.1, 'end' :5}
)]

print('Average app rating = ', np.mean(df['Rating']))
plotly.offline.iplot(data, filename='overall_rating_distribution')